# Convolutional Network

In [1]:
import os
import sys

sys.path.append(os.path.join(os.getcwd(), ".."))

In [2]:
from pyldl.modules import Flatten

In [3]:
import numpy as np

## Flatten Module

In [12]:
X = np.array([
    [
        [[1, 2], [3, 4]],
        [[5, 6], [7, 8]],
        [[9, 10], [11, 12]],
    ],
    [
        [[1, 2], [3, 4]],
        [[5, 6], [7, 8]],
        [[9, 10], [11, 12]],
    ],
    [
        [[1, 2], [3, 4]],
        [[5, 6], [7, 8]],
        [[9, 10], [11, 12]],
    ]
])

delta = np.ones_like(X)

flatten_module = Flatten()
output = flatten_module.forward(X)
print("Flattened input: ", output)
print("Output shape: ", output.shape)

backward_delta_output = flatten_module.backward_delta(X, delta)
print("\nDelta backward pass: ", backward_delta_output)
print("Delta original shape: ", delta.shape)
print("Output delta shape: ", backward_delta_output.shape)

Flattened input:  [[ 1  2  3  4  5  6  7  8  9 10 11 12]
 [ 1  2  3  4  5  6  7  8  9 10 11 12]
 [ 1  2  3  4  5  6  7  8  9 10 11 12]]
Output shape:  (3, 12)

Delta backward pass:  [[[[1 1]
   [1 1]]

  [[1 1]
   [1 1]]

  [[1 1]
   [1 1]]]


 [[[1 1]
   [1 1]]

  [[1 1]
   [1 1]]

  [[1 1]
   [1 1]]]


 [[[1 1]
   [1 1]]

  [[1 1]
   [1 1]]

  [[1 1]
   [1 1]]]]
Delta original shape:  (3, 3, 2, 2)
Output delta shape:  (3, 3, 2, 2)
